In [22]:
!pip install mlxtend


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [23]:
import pandas as pd
import numpy as np
import time
import scipy.sparse as sparse
from sklearn.metrics.pairwise import cosine_similarity
import os
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [2]:
df = pd.read_csv('/Users/Kate/Downloads/2020-Jan.csv')
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-01-01 00:00:00 UTC,view,5809910,1602943681873052386,NaN,grattol,5.24,595414620,4adb70bb-edbd-4981-b60f-a05bfd32683a
1,2020-01-01 00:00:09 UTC,view,5812943,1487580012121948301,NaN,kinetics,3.97,595414640,c8c5205d-be43-4f1d-aa56-4828b8151c8a
2,2020-01-01 00:00:19 UTC,view,5798924,1783999068867920626,NaN,zinger,3.97,595412617,46a5010f-bd69-4fbe-a00d-bb17aa7b46f3
3,2020-01-01 00:00:24 UTC,view,5793052,1487580005754995573,NaN,NaN,4.92,420652863,546f6af3-a517-4752-a98b-80c4c5860711
4,2020-01-01 00:00:25 UTC,view,5899926,2115334439910245200,NaN,NaN,3.92,484071203,cff70ddf-529e-4b0c-a4fc-f43a749c0acb


In [5]:
df.columns

Index(['event_time', 'event_type', 'product_id', 'category_id',
       'category_code', 'brand', 'price', 'user_id', 'user_session'],
      dtype='object')

In [4]:
purchase_df = df[df['event_type'] == 'purchase']
purchase_df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
60,2020-01-01 00:06:07 UTC,purchase,5802440,2151191070908613477,NaN,NaN,2.16,595411904,74ca1cd5-5381-4ffe-b00b-a258b390db77
61,2020-01-01 00:06:07 UTC,purchase,5825203,2089259162625114209,NaN,kiss,7.92,595411904,74ca1cd5-5381-4ffe-b00b-a258b390db77
590,2020-01-01 00:51:38 UTC,purchase,5842655,1487580013841613016,NaN,NaN,14.51,595351517,4ceb1e27-416e-4a3c-972a-03eb373d67b2
591,2020-01-01 00:52:00 UTC,purchase,4185,1487580005411062629,NaN,cnd,19.37,287758725,b276b96f-724f-4081-b56b-e8bc5e32e0c8
593,2020-01-01 00:52:11 UTC,purchase,5700088,1487580011677352062,NaN,NaN,10.95,504269664,32a5c608-4d22-475b-87d6-abb058930dbf


# COSINE SIMILARITY

In [7]:
orders = purchase_df[['user_session', 'product_id']]
orders['product_quantity'] = 1
orders.head()

/var/folders/g4/67myv6ld5mnd2mmg2jzxfpj40000gn/T/ipykernel_3356/2043821575.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['product_quantity'] = 1


,user_session,product_id,product_quantity
60,74ca1cd5-5381-4ffe-b00b-a258b390db77,5802440,1
61,74ca1cd5-5381-4ffe-b00b-a258b390db77,5825203,1
590,4ceb1e27-416e-4a3c-972a-03eb373d67b2,5842655,1
591,b276b96f-724f-4081-b56b-e8bc5e32e0c8,4185,1
593,32a5c608-4d22-475b-87d6-abb058930dbf,5700088,1


In [20]:
def recommendations(df,filename,rows='user_session',cols='product_id',quantity='product_quantity',top=11):

    #Creating the sparse matrix to see the frequency products occur together
    start_time = time.time()
    orders = list(sorted(set(df[rows])))
    products = list(sorted(set(df[cols])))
    quantity = list(df[quantity])

    rs = pd.Categorical(df[rows], categories=orders).codes
    cs = pd.Categorical(df[cols], categories=products).codes

    sparse_matrix = sparse.csr_matrix((quantity, (rs,cs)), shape=(len(orders),len(products)))

    matrix_size = sparse_matrix.shape[0]*sparse_matrix.shape[1] # Number of possible interactions in the matrix
    num_purchases = len(sparse_matrix.nonzero()[0]) # Number of items interacted with
    sparsity = round(100*(1 - (float(num_purchases)/matrix_size)),2) #Calculating sparsity

    print('Sparse matrix creation time: {} seconds.\nSparsity: {}'.format(round(time.time() - start_time,2), sparsity))

    #Calculating the cosine similarity of all products with each other
    start_time = time.time()
    similarities = cosine_similarity(sparse_matrix.T)
    df_sim = pd.DataFrame(similarities, index=products, columns=products)
    end_time = time.time()
    print("Cosine calculation time: {} seconds.".format(round(end_time - start_time,2)))


    #Creating the table of top 10 recommendations for every product
    start_time = time.time()
    df_match = pd.DataFrame(index=products, columns=[f'Rec {i}' for i in range(1, top)])
    df_score = pd.DataFrame(index=products, columns=[f'Score {i}' for i in range(1, top)])

   # df_match = pd.DataFrame(index=products,columns =  #place for the recommendations
   # ['Rec 0','Rec 1','Rec 2','Rec 3','Rec 4','Rec 5','Rec 6','Rec 7','Rec 8','Rec 9','Rec 10'])
   # df_score = pd.DataFrame(index=products,columns =  #place for the scores
   # ['Score 0','Score 1','Score 2','Score 3','Score 4','Score 5','Score 6','Score 7','Score 8','Score 9','Score 10'])

    for i in range(len(products)): #inserting into tables
        # Get the top recommendations and scores, limited by the available data
        top_recs = df_sim.iloc[:, i].sort_values(ascending=False)
        # Filter out the product itself
        top_recs = top_recs[top_recs.index != df_sim.index[i]]
        # Ensuring num_recs is within the bounds of top_recs and df_match
        num_recs = min(top -1 , len(top_recs) -1, df_match.shape[1])

        df_match.iloc[i, :num_recs] = top_recs.iloc[:num_recs].index # Using iloc to select from top_recs
        df_score.iloc[i, :num_recs] = top_recs.iloc[:num_recs].values # Using iloc to select from top_recs

    for i in range(top-1): #remove recs with similarities of 0 # Adjusting loop range
        try:
            df_match.iloc[np.where(df_score.iloc[:,i] == 0),i] = None
            df_score.iloc[np.where(df_score.iloc[:,i] == 0),i] = None
        except:
            continue

    mismatch_loc = np.where(df_match.iloc[:,0] != df_match.index)[0] #Correcting sorting errors
    df_match.iloc[mismatch_loc,1] = df_match.iloc[mismatch_loc,0]

    df_new = df_match.merge(df_score, how='inner', left_index=True, right_index=True)
    # Removed the drop command as 'Rec 0' and 'Score 0' are no longer created
    # df_new.drop(['Rec 0','Score 0'],axis = 1,inplace=True)
    df_new.index.names = ['product_id']
    print('Table creation time: {} seconds'.format(round(time.time() - start_time,2)))

    #df_new.to_csv(filename)


    print('Process completed')
    return df_new

In [21]:
prod_recs = recommendations(orders, 'product_recs_cosine_similarity.csv')
prod_recs.head()

Sparse matrix creation time: 0.11 seconds.
Sparsity: 99.97
Cosine calculation time: 0.51 seconds.
Table creation time: 20.91 seconds
Process completed


,Rec 1,Rec 2,Rec 3,Rec 4,Rec 5,Rec 6,Rec 7,Rec 8,Rec 9,Rec 10,Score 1,Score 2,Score 3,Score 4,Score 5,Score 6,Score 7,Score 8,Score 9,Score 10
product_id,,,,,,,,,,,,,,,,,,,,
3762,4185,4185,5645618,5809326,5811584,5891943,5877588,6483,5804296,5804314,0.625543,0.180579,0.13901,0.120386,0.120386,0.120386,0.120386,0.120386,0.120386,0.120386
3774,18083,18083,None,None,None,None,None,None,None,None,1.0,0.707107,None,None,None,None,None,None,None,None
3806,5897540,5897540,7786,5842196,7785,33959,5804304,37628,5590822,5717141,0.516398,0.447214,0.447214,0.447214,0.447214,0.447214,0.447214,0.338062,0.316228,0.298142
3928,37628,37628,33956,8006,7786,5804289,5749233,5804303,5663276,5804270,0.507093,0.447214,0.447214,0.447214,0.447214,0.447214,0.4,0.365148,0.338062,0.316228
3936,5663273,5663273,5706780,5809899,5739901,5736284,5855282,4938,5809910,None,0.57735,0.258199,0.154303,0.149071,0.075165,0.056888,0.047619,0.037905,0.01253,None


# APRIORI

In [35]:
# Can still use the orders dataframe created a bove
basket = (orders
          .groupby(['user_session', 'product_id'])['product_quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('user_session'))  # Resets the index to keep 'user_session' as a column

basket.head()

product_id,3762,3774,3806,3928,3936,3978,4006,4185,4203,4210,...,5923738,5923970,5924029,5924037,5924043,5924044,5924045,5924047,5924066,5924352
user_session,,,,,,,,,,,,,,,,,,,,,
0004e273-eebe-488c-9189-d838568db29c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0005e389-5d8a-42d1-8df5-29d981d36efd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00112956-671f-444a-b8af-e9a03a9e2c42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0013b593-e8eb-4bcf-b117-441db7cd9c6d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001718ea-9be6-4356-9b33-c9a2a3ec74d0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
basket_sets = basket.applymap(encode_units)
basket_sets = basket_sets.astype(bool)

In [ ]:
# Generate frequent itemsets
frequent_itemsets = apriori(basket_sets, min_support=0.001, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1, num_itemsets=len(frequent_itemsets))
rules

In [ ]:
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets = frequent_itemsets[frequent_itemsets['length'] >=2]

# Sort the frequent itemsets by their support (descending) to get the most frequent ones
frequent_itemsets.sort_values(by='support', ascending=False)